In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']

In [3]:
fn = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv' %(subnum, subnum, runnum))
events = pd.read_csv(fn, sep='\t')

# Read in behavioral data with modeled value and RPE estimates
behavior = pd.read_csv(behavior_path)

# Extract the correct subnum and runnum from behavioral data
run_behavior = behavior.query('subnum == %d & session == %d'%(int(subnum), int(runnum)))

# Demean columns that might be used for parametric regressors
demean_cols = ['probFractalDraw', 'leftBundleVal', 'rightBundleVal', 'leftLotteryEV', 'rightLotteryEV', 'leftQValue', 'rightQValue', 'reward', 'rpe', 'leftQVAdv', 'leftEVAdv', 'leftbundleValAdv', 'leftFractalRpe', 'rightFractalRpe']
for cur_col in demean_cols:
    run_behavior.loc[:,cur_col].sub(run_behavior[cur_col].mean())

In [18]:
np.array(np.where(run_behavior['probFractalDraw']<0.4, 1,0) * run_behavior['leftEVAdv'])
# np.where( (run_behavior['probFractalDraw']>0.4) & (run_behavior['probFractalDraw']<0.7))

array([-0. ,  0. ,  0. , -0. , -0. ,  0. ,  0. ,  0.4,  0.4,  0. , -0.4,
       -0.4,  0. ,  0. ,  0. , -0.2,  0. ,  0. , -0.4,  0. ,  0. , -0. ,
       -0. , -0.4, -0. ,  0.2, -0. ,  0. , -0. , -0. , -0. ,  0. ,  0. ,
        0. , -0.2,  0.2,  0. ,  0.2,  0. ,  0. ,  0.4,  0. ,  0. ,  0.2,
        0. ,  0. , -0. ,  0. , -0.4,  0. , -0. ,  0. ,  0. , -0.2,  0.2,
       -0.2, -0. , -0.4,  0. , -0. ])

In [12]:
np.where(run_behavior['probFractalDraw']<0.4)

(array([ 5,  7,  8, 10, 11, 15, 18, 23, 25, 34, 35, 36, 37, 38, 40, 43, 48,
        53, 54, 55, 57]),)

In [11]:
np.where(run_behavior['probFractalDraw']>0.6)

(array([ 2,  4,  6, 13, 16, 17, 22, 26, 27, 28, 29, 30, 31, 32, 44, 45, 46,
        50, 52, 58, 59]),)

In [25]:
cond_valDiffLotteryLowPFrac_par = events.query('trial_type == "stimulus"')[['onset', 'duration']].reset_index(drop=True)
cond_valDiffLotteryLowPFrac_par['trial_type'] = 'valDiffLotteryLowPFrac_par'
cond_valDiffLotteryLowPFrac_par['modulation'] = np.array(np.where(run_behavior['probFractalDraw']<0.4, 1,0) * run_behavior['leftEVAdv'])
cond_valDiffLotteryLowPFrac_par

,onset,duration,trial_type,modulation
0,8.060577,1.340867,valDiffLotteryLowPFrac_par,-0.0
1,22.133918,0.892700,valDiffLotteryLowPFrac_par,0.0
2,36.207245,0.732448,valDiffLotteryLowPFrac_par,0.0
3,52.314899,1.035707,valDiffLotteryLowPFrac_par,-0.0
4,65.387724,0.626727,valDiffLotteryLowPFrac_par,-0.0
5,81.478734,1.001806,valDiffLotteryLowPFrac_par,0.0
6,94.568253,0.338375,valDiffLotteryLowPFrac_par,0.0
7,109.658746,0.866059,valDiffLotteryLowPFrac_par,0.4
8,122.731607,1.055714,valDiffLotteryLowPFrac_par,0.4
9,135.837830,1.177396,valDiffLotteryLowPFrac_par,0.0


In [24]:
cond_valDiffLotteryMedPFrac_par = events.query('trial_type == "stimulus"')[['onset', 'duration']].reset_index(drop=True)
cond_valDiffLotteryMedPFrac_par['trial_type'] = 'valDiffLotteryMedPFrac_par'
cond_valDiffLotteryMedPFrac_par['modulation'] = np.array(np.where( (run_behavior['probFractalDraw']>0.4) & (run_behavior['probFractalDraw']<0.7), 1,0) * run_behavior['leftEVAdv'])
cond_valDiffLotteryMedPFrac_par

,onset,duration,trial_type,modulation
0,8.060577,1.340867,valDiffLotteryMedPFrac_par,-0.2
1,22.133918,0.892700,valDiffLotteryMedPFrac_par,0.4
2,36.207245,0.732448,valDiffLotteryMedPFrac_par,0.0
3,52.314899,1.035707,valDiffLotteryMedPFrac_par,-0.0
4,65.387724,0.626727,valDiffLotteryMedPFrac_par,-0.0
5,81.478734,1.001806,valDiffLotteryMedPFrac_par,0.0
6,94.568253,0.338375,valDiffLotteryMedPFrac_par,0.0
7,109.658746,0.866059,valDiffLotteryMedPFrac_par,0.0
8,122.731607,1.055714,valDiffLotteryMedPFrac_par,0.0
9,135.837830,1.177396,valDiffLotteryMedPFrac_par,0.2


In [ ]:
if regress_rt:
    cond_valDiffFractalWeighted_par = events.query('trial_type == "stimulus"')[['onset']].reset_index(drop=True)
    cond_valDiffFractalWeighted_par['duration'] = mean_rt
else:
    cond_valDiffFractalWeighted_par = events.query('trial_type == "stimulus"')[['onset', 'duration']].reset_index(drop=True)
cond_valDiffFractalWeighted_par['trial_type'] = 'valDiffFractalWeighted_par'
cond_valDiffFractalWeighted_par['modulation'] = np.array(run_behavior['leftQVAdv']*run_behavior['wpFrac'])

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model10a', data_path, behavior_path, regress_rt = 1)
plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 1)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt))
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)